In [ ]:
import os
import re
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)


In [ ]:
from src.data_utils import load_and_prepare_data
from src.next_token_dataset import TextDataset
from src.lstm_model import LSTMModel

In [ ]:
df, tokenizer, tokens = load_and_prepare_data(data_dir="data", max_len=20, min_len=7, use_sample=True)

print("Пример первых строк датасета:")
print(df.head())
print("Пример токенов:")
print(tokens['input_ids'][:2])


In [ ]:

def create_examples(tokens_ids):
    X_list, Y_list = [], []
    for token in tokens_ids:
        X = token[:-1]
        Y = token[1:]
        X_list.append(torch.tensor(X))
        Y_list.append(torch.tensor(Y))
    return X_list, Y_list

token_ids = tokens['input_ids'].tolist()
train_ids, temp_ids = train_test_split(token_ids, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

X_train, Y_train = create_examples(train_ids)


scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Функция генерации последовательности
def generate_sequence(model, start_seq, max_len=20):
    model.eval()
    generated = start_seq.tolist()
    input_seq = start_seq.unsqueeze(0).to(device)
    hidden = None
    for _ in range(max_len):
        logits, hidden = model(input_seq, hidden)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        generated.append(next_token.item())
        input_seq = torch.cat([input_seq, next_token.unsqueeze(0)], dim=1)
    return generated

# Тренировка одной эпохи
def train_epoch(model, loader):
    model.train()
    total_loss = 0
    for X, Y in tqdm(loader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        logits, _ = model(X)
        loss = criterion(logits.view(-1, vocab_size), Y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Оценка ROUGE
def evaluate(model, loader):
    model.eval()
    scores = []
    with torch.no_grad():
        for X, Y in loader:
            X, Y = X.to(device), Y.to(device)
            for i in range(X.size(0)):
                seq_input = X[i][:int(0.75*X.size(1))]
                target_seq = Y[i][int(0.75*Y.size(1)):]
                pred_tokens = generate_sequence(model, seq_input, max_len=len(target_seq))
                pred_text = tokenizer.decode(pred_tokens[-len(target_seq):])
                target_text = tokenizer.decode(target_seq.tolist())
                score = scorer.score(pred_text, target_text)['rougeL'].fmeasure
                scores.append(score)
    return sum(scores)/len(scores)

X_val, Y_val = create_examples(val_ids)
X_test, Y_test = create_examples(test_ids)

train_dataset = TextDataset(X_train, Y_train)
val_dataset = TextDataset(X_val, Y_val)
test_dataset = TextDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Размеры выборок:", len(train_dataset), len(val_dataset), len(test_dataset))


In [ ]:

vocab_size = tokenizer.vocab_size
model = LSTMModel(vocab_size, embed_dim=128, hidden_dim=128, num_layers=1).to(device)
print(model)


In [ ]:

# Этап 3: Тренировка LSTM

for epoch in range(2):
    loss = train_epoch(model, train_loader)
    rouge_score = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, ROUGE-L={rouge_score:.4f}")


In [ ]:

# Этап 4: Предобученный трансформер
tokenizer_gpt = AutoTokenizer.from_pretrained("distilgpt2")
model_gpt = AutoModelForCausalLM.from_pretrained("distilgpt2").to(device)

def generate_gpt2(text, max_new_tokens=20):
    inputs = tokenizer_gpt(text, return_tensors="pt").to(device)
    outputs = model_gpt.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.8
    )
    return tokenizer_gpt.decode(outputs[0], skip_special_tokens=True)

# Проверка на первых 50 примерах
scores = []
for text in df['text'][:50]:
    input_text = ' '.join(text.split()[:int(len(text.split())*0.75)])
    target_text = ' '.join(text.split()[int(len(text.split())*0.75):])
    pred_text = generate_gpt2(input_text)
    score = scorer.score(pred_text, target_text)['rougeL'].fmeasure
    scores.append(score)

print("Average ROUGE-L for GPT2:", sum(scores)/len(scores))


In [ ]:

# Этап 5: Выводы
print("Сравнение моделей:")
print("- LSTM: ROUGE-L на валидации (после 2 эпох):", rouge_score)
print("- DistilGPT2: ROUGE-L на первых 50 примерах:", sum(scores)/len(scores))


**Сравнение моделей:**

LSTM:

- Быстрее обучение
- Меньше требований к памяти
- Стабильная сходимость
- Transformer пострадал от сокращения:


Transformer:
- Недостаточно данных
- Низкое качество
- Требует полного датасета для раскрытия потенциала
- Ключевые выводы

LSTM показала лучшие результаты на сокращенных данных:

Высокое качество (ROUGE-L: 0.346 vs 0.045) Эффективность на малых данных - главное преимущество Быстрая сходимость - хорошие результаты после 2 эпох Устойчивость к ограничениям ресурсов